# Exercises for "Hands-on with Pydata: How to Build a Minimal Recommendation Engine"

# Systems check: imports and files

In [1]:
import numpy as np
import pandas as pd

# Pandas questions: Series and DataFrames
## 1. Adding a column in a DataFrame

In [2]:
# given the following DataFrame, add a new column to it
df = pd.DataFrame({'col1': [1,2,3,4]})
df

col1
0     1
1     2
2     3
3     4

In [3]:
df = pd.concat([df,pd.DataFrame({'col2':[5,6,7,8]})], axis=1)
df
# alternate: df['r_3'] = ['F', 'Drama', 1193, 5, 978300760, 1]

col1  col2
0     1     5
1     2     6
2     3     7
3     4     8

## 2. Deleting a row in a DataFrame

In [4]:
# given the following DataFrame, delete row 'd' from it
df = pd.DataFrame({'col1': [1,2,3,4]}, index = ['a','b','c','d'])
df

col1
a     1
b     2
c     3
d     4

In [5]:
df = df.drop('d')
df

col1
a     1
b     2
c     3

## 3. Creating a DataFrame from a few Series

In [6]:
# given the following three Series, create a DataFrame such that it holds them as its columns
ser_1 = pd.Series(np.random.randn(6))
ser_2 = pd.Series(np.random.randn(6))
ser_3 = pd.Series(np.random.randn(6))

df123 = pd.DataFrame({'ser1': ser_1,'ser2':ser_2, 'ser3':ser_3})
df123
#alternate way: using concat as I did earlier
# look at https://stackoverflow.com/questions/18062135/combining-two-series-into-a-dataframe-in-pandas for example

ser1      ser2      ser3
0 -0.879172  1.138693 -0.872078
1 -0.403653  0.433790  0.790313
2 -1.387465 -0.140473 -0.832569
3  0.116737 -0.187478  0.660340
4 -0.705763  0.836846  0.119289
5  0.411293 -0.559695  0.266535

# Pandas questions: Indexing

## 1. Indexing into a specific column

In [7]:
# given the following DataFrame, try to index into the 'col_2' column
df = pd.DataFrame(data={'col_1': [0.12, 7, 45, 10], 'col_2': [0.9, 9, 34, 11]},
                  columns=['col_1', 'col_2', 'col_3'],
                  index=['obs1', 'obs2', 'obs3', 'obs4'])
df

col_1  col_2 col_3
obs1   0.12    0.9   NaN
obs2   7.00    9.0   NaN
obs3  45.00   34.0   NaN
obs4  10.00   11.0   NaN

In [8]:
df['col_2'] #also note df.col_2 would work

obs1     0.9
obs2     9.0
obs3    34.0
obs4    11.0
Name: col_2, dtype: float64

## 2. Label-based indexing

In [9]:
# using the same DataFrame, index into the row whose index is 'obs3'
df.ix['obs3'] # could also use df.loc['obs3']

col_1     45
col_2     34
col_3    NaN
Name: obs3, dtype: object

## 3. Position-based indexing

In [10]:
# using the same DataFrame, index into into its first row
df.ix[0]
# could also use df.iloc[0]

col_1    0.12
col_2     0.9
col_3     NaN
Name: obs1, dtype: object

# Mini-Challenge prep: data loading

## 1. How to load the `users` and `movies` portions of MovieLens

In [11]:
import pandas as pd

users = pd.read_table('data/ml-1m/users.dat',
                      sep='::', header=None,
                      names=['user_id', 'gender', 'age', 'occupation', 'zip'], engine = 'python')

movies = pd.read_table('data/ml-1m/movies.dat',
                       sep='::', header=None,
                       names=['movie_id', 'title', 'genres'], engine = 'python')
movies.head()

movie_id                               title                        genres
0         1                    Toy Story (1995)   Animation|Children's|Comedy
1         2                      Jumanji (1995)  Adventure|Children's|Fantasy
2         3             Grumpier Old Men (1995)                Comedy|Romance
3         4            Waiting to Exhale (1995)                  Comedy|Drama
4         5  Father of the Bride Part II (1995)                        Comedy

## 2. How to load the training and testing subsets

In [12]:
# subset version (hosted notebook)
movielens_train = pd.read_csv('data/movielens_train.csv', index_col=0,encoding='latin-1') # the enconding 'latin' handled a utf-8 error
movielens_test = pd.read_csv('data/movielens_test.csv', index_col=0,encoding='latin-1')

In [13]:
movielens_train.head()

user_id  movie_id  rating  timestamp gender  age  occupation    zip  \
593263     3562      3798       4  967332344      F   25           6  32812   
235597     1051      3793       4  974958593      F   25           0  60513   
219003     3727      2366       3  966309522      M   35           7  74401   
685090     4666      1094       3  963843918      M   35           1  53704   
312377     3261      1095       4  968251750      M   45          20  87505   

                             title                   genres for_testing  
593263    What Lies Beneath (2000)                 Thriller       False  
235597                X-Men (2000)            Action|Sci-Fi       False  
219003            King Kong (1933)  Action|Adventure|Horror       False  
685090     Crying Game, The (1992)        Drama|Romance|War       False  
312377  Glengarry Glen Ross (1992)                    Drama       False

In [14]:
movielens_test.head()

user_id  movie_id  rating   timestamp gender  age  occupation    zip  \
693323     4653      2648       4   975532459      M   35          12  95051   
24177      2259      1270       4   974591524      F   56          16  70503   
202202     3032      1378       5   970343147      M   25           0  47303   
262003     3029      2289       4   972846393      M   18           4  92037   
777848     4186      2403       3  1017931262      M   25           7  33308   

                            title                 genres for_testing  
693323        Frankenstein (1931)                 Horror       False  
24177   Back to the Future (1985)          Comedy|Sci-Fi       False  
202202          Young Guns (1988)  Action|Comedy|Western       False  
262003         Player, The (1992)           Comedy|Drama       False  
777848         First Blood (1982)                 Action       False

# Mini-Challenge prep: evaluation functions

These are the two functions that you will need to test your `estimate` method.

In [18]:
def compute_rmse(y_pred, y_true):
    """ Compute Root Mean Squared Error. """
    
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

In [16]:
def evaluate(estimate_f):
    """ RMSE-based predictive performance evaluation with pandas. """
    
    ids_to_estimate = zip(movielens_test.user_id, movielens_test.movie_id)
    estimated = np.array([estimate_f(u,i) for (u,i) in ids_to_estimate])
    real = movielens_test.rating.values
    return compute_rmse(estimated, real)

Test a dummy solution!

In [17]:
def my_estimate_func(user_id, movie_id):
    return 3.0

You can test for performance with the following line, which assumes that your function is called `my_estimate_func`:

In [19]:
print('RMSE for my estimate function: %s' % evaluate(my_estimate_func))

RMSE for my estimate function: 1.23237195265


# Reco systems questions: Minimal reco engine v1.0

## 1. Simple collaborative filtering using mean ratings

In [20]:
# write an 'estimate' function that computes the mean rating of a particular user
def collab_mean(user_id, movie_id):
    movie_condition = movielens_train.movie_id == movie_id # first, index into all ratings of this movie
    mov_avg = movielens_train.loc[movie_condition, 'rating'].mean() # second, compute the mean of those ratings
    
    return mov_avg

    
# try it out for a user_id, movie_id pair
collab_mean(4653, 2648)

4.0

In [26]:
# write an 'estimate' function that computes the mean rating of a particular user
def collab_mean(user_id, movie_id):
    """Simple collaborative filter basked on mean ratings."""
    
    user_condition = movielens_train.user_id != user_id
    movie_condition = movielens_train.movie_id == movie_id 
    ratings_by_others = movielens_train.loc[user_condition & movie_condition] 
    if ratings_by_others.empty:
        return 3.0
    else:
        return ratings_by_others.rating.mean()

#print('RMSE for estimate2: %s') % evaluate(collab_mean)

In [27]:
evaluate(collab_mean)

1.1234279896011794

# Mini-Challenge: first round
Implement an `estimate` function of your own using other similarity notions, eg.:

- collaborative filter based on age similarities
- collaborative filter based on zip code similarities
- collaborative filter based on occupation similarities
- content filter based on movie genre

In [28]:
movielens_train.groupby('gender')['rating'].mean()

gender
F    3.594928
M    3.530507
Name: rating, dtype: float64

In [29]:
movielens_train.groupby(['gender', 'age'])['rating'].mean()

gender  age
F       1      3.500000
        18     3.528958
        25     3.548507
        35     3.730104
        45     3.581818
        50     3.617978
        56     3.725490
M       1      3.305556
        18     3.507712
        25     3.489764
        35     3.569591
        45     3.565574
        50     3.728125
        56     3.611111
Name: rating, dtype: float64

In [30]:
movielens_train.pivot_table(values='rating', index='age', columns='gender', aggfunc='mean')

gender         F         M
age                       
1       3.500000  3.305556
18      3.528958  3.507712
25      3.548507  3.489764
35      3.730104  3.569591
45      3.581818  3.565574
50      3.617978  3.728125
56      3.725490  3.611111

In [31]:
movielens_train.pivot_table(values='rating', index='age', columns='gender', aggfunc=[np.mean, np.std])

mean                 std          
gender         F         M         F         M
age                                           
1       3.500000  3.305556  1.242757  1.335765
18      3.528958  3.507712  1.162283  1.151606
25      3.548507  3.489764  1.146094  1.101005
35      3.730104  3.569591  0.984159  1.112843
45      3.581818  3.565574  1.183850  1.082775
50      3.617978  3.728125  1.049953  1.009899
56      3.725490  3.611111  0.981396  1.073106

In [32]:
user_info = users.set_index('user_id')
user_info.head(5)

gender  age  occupation    zip
user_id                               
1            F    1          10  48067
2            M   56          16  70072
3            M   25          15  55117
4            M   45           7  02460
5            M   25          20  55455

In [33]:
user_id = 3
user_info.loc[user_id, 'gender'] #getting the gender of a specific user

'M'

In [35]:
#collaborative filter based on gender similarities
def collab_gender(user_id, movie_id):
    """ Collaborative filtering using an implicit sim(u,u') based on gender. """
    
    user_condition = movielens_train.user_id != user_id
    movie_condition = movielens_train.movie_id == movie_id
    ratings_by_others = movielens_train.loc[user_condition & movie_condition]
    if ratings_by_others.empty: 
        return 3.0
    
    means_by_gender = ratings_by_others.pivot_table('rating', index='movie_id', columns='gender')
    user_gender = user_info.ix[user_id, 'gender']
    if user_gender in means_by_gender.columns: 
        return means_by_gender.ix[movie_id, user_gender]
    else:
        return means_by_gender.ix[movie_id].mean()

print('RMSE for collab_gender: %s' % evaluate(collab_gender))

RMSE for collab_gender: 1.17400824171


In [36]:
class CollabGenderReco:
    """ Collaborative filtering using an implicit sim(u,u'). """

    def learn(self):
        """ Prepare datastructures for estimation. """
        
        self.means_by_gender = movielens_train.pivot_table('rating', index='movie_id', columns='gender')

    def estimate(self, user_id, movie_id):
        """ Mean ratings by other users of the same gender. """
        
        if movie_id not in self.means_by_gender.index: 
            return 3.0
        
        user_gender = user_info.ix[user_id, 'gender']
        if ~np.isnan(self.means_by_gender.ix[movie_id, user_gender]):
            return self.means_by_gender.ix[movie_id, user_gender]
        else:
            return self.means_by_gender.ix[movie_id].mean()

reco = CollabGenderReco()
reco.learn()
print('RMSE for CollabGenderReco: %s' % evaluate(reco.estimate))

RMSE for CollabGenderReco: 1.17400824171


Using this second method was much faster for computation time.

In [37]:
#collaborative filter based on age similarities
class CollabAgeReco:
    """ Collaborative filtering using an implicit sim(u,u'). """

    def learn(self):
        """ Prepare datastructures for estimation. """
        
        self.means_by_age = movielens_train.pivot_table('rating', index='movie_id', columns='age')

    def estimate(self, user_id, movie_id):
        """ Mean ratings by other users of the same age. """
        
        if movie_id not in self.means_by_age.index: 
            return 3.0
        
        user_age = user_info.ix[user_id, 'age']
        if ~np.isnan(self.means_by_age.ix[movie_id, user_age]):
            return self.means_by_age.ix[movie_id, user_age]
        else:
            return self.means_by_age.ix[movie_id].mean()

reco = CollabAgeReco()
reco.learn()
print('RMSE for CollabAgeReco: %s' % evaluate(reco.estimate))

RMSE for CollabAgeReco: 1.20520133441


In [38]:
#collaborative filter based on zip code similarities
class CollabZipReco:
    """ Collaborative filtering using an implicit sim(u,u'). """

    def learn(self):
        """ Prepare datastructures for estimation. """
        
        self.means_by_zip = movielens_train.pivot_table('rating', index='movie_id', columns='zip')

    def estimate(self, user_id, movie_id):
        """ Mean ratings by other users of the same zip code. """
        
        if movie_id not in self.means_by_zip.index: 
            return 3.0
        
        user_zip = user_info.ix[user_id, 'zip']
        if ~np.isnan(self.means_by_zip.ix[movie_id, user_zip]):
            return self.means_by_zip.ix[movie_id, user_zip]
        else:
            return self.means_by_zip.ix[movie_id].mean()

reco = CollabZipReco()
reco.learn()
print('RMSE for CollabZipReco: %s' % evaluate(reco.estimate))

RMSE for CollabZipReco: 1.12566403192


In [39]:
#collaborative filter based on occupation similarities
class CollabJobReco:
    """ Collaborative filtering using an implicit sim(u,u'). """

    def learn(self):
        """ Prepare datastructures for estimation. """
        
        self.means_by_job = movielens_train.pivot_table('rating', index='movie_id', columns='occupation')

    def estimate(self, user_id, movie_id):
        """ Mean ratings by other users of the same occupation. """
        
        if movie_id not in self.means_by_job.index: 
            return 3.0
        
        user_job = user_info.ix[user_id, 'occupation']
        if ~np.isnan(self.means_by_job.ix[movie_id, user_job]):
            return self.means_by_job.ix[movie_id, user_job]
        else:
            return self.means_by_job.ix[movie_id].mean()

reco = CollabJobReco()
reco.learn()
print('RMSE for CollabJobReco: %s' % evaluate(reco.estimate))

RMSE for CollabJobReco: 1.20287696436


In [ ]:
#content filter based on movie genre
def content_mean(user_id, movie_id):
    """ Simple content-filtering based on mean ratings. """
    
    user_condition = movielens_train.user_id == user_id
    return movielens_train.loc[user_condition, 'rating'].mean()

print 'RMSE for estimate1: %s' % evaluate(content_mean)

class ContentGenreReco:
    """ Content filtering using an implicit sim(u,u'). """

    def learn(self):
        """ Prepare datastructures for estimation. """
        
        self.means_by_job = movielens_train.pivot_table('rating', index='movie_id', columns='occupation')

    def estimate(self, user_id, movie_id):
        """ Mean ratings by other users of the same occupation. """
        
        if movie_id not in self.means_by_job.index: 
            return 3.0
        
        user_job = user_info.ix[user_id, 'occupation']
        if ~np.isnan(self.means_by_job.ix[movie_id, user_job]):
            return self.means_by_job.ix[movie_id, user_job]
        else:
            return self.means_by_job.ix[movie_id].mean()

reco = CollabJobReco()
reco.learn()
print('RMSE for CollabJobReco: %s' % evaluate(reco.estimate))

# Mini-Challenge: second round
Implement an `estimate` function of your own using other custom similarity notions, eg.:

- euclidean
- cosine

In [40]:
def euclidean(s1, s2):
    """Take two pd.Series objects and return their euclidean 'similarity'."""
    diff = s1 - s2
    return 1 / (1 + np.sqrt(np.sum(diff ** 2)))

In [41]:
def cosine(s1, s2):
    """Take two pd.Series objects and return their cosine similarity."""
    return np.sum(s1 * s2) / np.sqrt(np.sum(s1 ** 2) * np.sum(s2 ** 2))

In [42]:
def pearson(s1, s2):
    """Take two pd.Series objects and return a pearson correlation."""
    s1_c = s1 - s1.mean()
    s2_c = s2 - s2.mean()
    return np.sum(s1_c * s2_c) / np.sqrt(np.sum(s1_c ** 2) * np.sum(s2_c ** 2))

In [43]:
def jaccard(s1, s2):
    dotp = np.sum(s1 * s2)
    return dotp / (np.sum(s1 ** 2) + np.sum(s2 ** 2) - dotp)

def binjaccard(s1, s2):
    dotp = (s1.index & s2.index).size
    return dotp / (s1.sum() + s2.sum() - dotp)

In [45]:
class CollabPearsonReco:
    """ Collaborative filtering using a custom sim(u,u'). """

    def learn(self):
        """ Prepare datastructures for estimation. """
        
        self.all_user_profiles = movielens_train.pivot_table('rating', index='movie_id', columns='user_id')

    def estimate(self, user_id, movie_id):
        """ Ratings weighted by correlation similarity. """
        
        user_condition = movielens_train.user_id != user_id
        movie_condition = movielens_train.movie_id == movie_id
        ratings_by_others = movielens_train.loc[user_condition & movie_condition]
        if ratings_by_others.empty: 
            return 3.0
        
        ratings_by_others.set_index('user_id', inplace=True)
        their_ids = ratings_by_others.index
        their_ratings = ratings_by_others.rating
        their_profiles = self.all_user_profiles[their_ids]
        user_profile = self.all_user_profiles[user_id]
        sims = their_profiles.apply(lambda profile: pearson(profile, user_profile), axis=0)
        ratings_sims = pd.DataFrame({'sim': sims, 'rating': their_ratings})
        ratings_sims = ratings_sims[ratings_sims.sim > 0]
        if ratings_sims.empty:
            return their_ratings.mean()
        else:
            return np.average(ratings_sims.rating, weights=ratings_sims.sim)
        
reco = CollabPearsonReco()
reco.learn()
print('RMSE for CollabPearsonReco: %s' % evaluate(reco.estimate))

C:\Users\asduser\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: RuntimeWarning: invalid value encountered in double_scalars


RMSE for CollabPearsonReco: 1.12640340009


In [47]:
class CollabEuclidReco:
    """ Collaborative filtering using a custom sim(u,u'). """

    def learn(self):
        """ Prepare datastructures for estimation. """
        
        self.all_user_profiles = movielens_train.pivot_table('rating', index='movie_id', columns='user_id')

    def estimate(self, user_id, movie_id):
        """ Ratings weighted by correlation similarity. """
        
        user_condition = movielens_train.user_id != user_id
        movie_condition = movielens_train.movie_id == movie_id
        ratings_by_others = movielens_train.loc[user_condition & movie_condition]
        if ratings_by_others.empty: 
            return 3.0
        
        ratings_by_others.set_index('user_id', inplace=True)
        their_ids = ratings_by_others.index
        their_ratings = ratings_by_others.rating
        their_profiles = self.all_user_profiles[their_ids]
        user_profile = self.all_user_profiles[user_id]
        sims = their_profiles.apply(lambda profile: euclidean(profile, user_profile), axis=0)
        ratings_sims = pd.DataFrame({'sim': sims, 'rating': their_ratings})
        ratings_sims = ratings_sims[ratings_sims.sim > 0]
        if ratings_sims.empty:
            return their_ratings.mean()
        else:
            return np.average(ratings_sims.rating, weights=ratings_sims.sim)
        
reco = CollabEuclidReco()
reco.learn()
print('RMSE for CollabEuclidReco: %s' % evaluate(reco.estimate))

RMSE for CollabEuclidReco: 1.123429476


In [48]:
class CollabCosineReco:
    """ Collaborative filtering using a custom sim(u,u'). """

    def learn(self):
        """ Prepare datastructures for estimation. """
        
        self.all_user_profiles = movielens_train.pivot_table('rating', index='movie_id', columns='user_id')

    def estimate(self, user_id, movie_id):
        """ Ratings weighted by correlation similarity. """
        
        user_condition = movielens_train.user_id != user_id
        movie_condition = movielens_train.movie_id == movie_id
        ratings_by_others = movielens_train.loc[user_condition & movie_condition]
        if ratings_by_others.empty: 
            return 3.0
        
        ratings_by_others.set_index('user_id', inplace=True)
        their_ids = ratings_by_others.index
        their_ratings = ratings_by_others.rating
        their_profiles = self.all_user_profiles[their_ids]
        user_profile = self.all_user_profiles[user_id]
        sims = their_profiles.apply(lambda profile: cosine(profile, user_profile), axis=0)
        ratings_sims = pd.DataFrame({'sim': sims, 'rating': their_ratings})
        ratings_sims = ratings_sims[ratings_sims.sim > 0]
        if ratings_sims.empty:
            return their_ratings.mean()
        else:
            return np.average(ratings_sims.rating, weights=ratings_sims.sim)
        
reco = CollabCosineReco()
reco.learn()
print('RMSE for CollabCosineReco: %s' % evaluate(reco.estimate))

RMSE for CollabCosineReco: 1.13439921696
